# Advent of Code 2024

## Contents
<table>
<tr><td>

- [Day 1](#day-1)
- [Day 2](#day-2)
- [Day 3](#day-3)
- [Day 4](#day-4)
- [Day 5](#day-5)

</td><td>

- [Day 6](#day-6)
- [Day 7](#day-7)
- [Day 8](#day-8)
- [Day 9](#day-9)
- [Day 10](#day-10)

</td><td>

- [Day 11](#day-11)
- [Day 12](#day-12)
- [Day 13](#day-13)
- [Day 14](#day-14)
- [Day 15](#day-15)

</td><td>

- [Day 16](#day-16)
- [Day 17](#day-17)
- [Day 18](#day-18)
- [Day 19](#day-19)
- [Day 20](#day-20)

</td><td>

- [Day 21](#day-21)
- [Day 22](#day-22)
- [Day 23](#day-23)
- [Day 24](#day-24)
- [Day 25](#day-25)

</td></tr>
</table>

## Boilerplate

In [1]:
# SETUP #
# Currently just what was needed in 2023, will adjust as we go
import sys
import math
import operator
import copy
import re
import numpy as np
import cProfile
from itertools import compress, combinations, product
from functools import reduce, cmp_to_key, cache
from dataclasses import dataclass
from typing import Tuple, List, Dict, TypeVar
from collections import Counter, defaultdict

T = TypeVar('T')

@dataclass
class DayData:
    input: str
    test_input: str
    test_solutions: Tuple[int, int]

# Load input and solution data #
def init_day(day, test_solutions) -> DayData:

    def read_file(path):
        with open(path, mode="rt") as f:
            return f.read()

    return DayData(
        input = read_file(f"inputs/{day}.txt"),
        test_input = read_file(f"test_inputs/{day}.txt"),
        test_solutions = test_solutions
    )

# Run and test code #
def run_test(func_to_run, day_data):
    def test(test: int, solution: int):
        return "Success!" if test == solution else f"Failed. Expected {solution}, but got {test}."

    print(f"Test Part 1: {test(func_to_run(1,day_data.test_input), day_data.test_solutions[0])}")
    print(f"Test Part 2: {test(func_to_run(2,day_data.test_input), day_data.test_solutions[1])}")

def run_real(func_to_run, day_data):
    %time print(f"Answer Part 1: {func_to_run(1,day_data.input)}")
    %time print(f"Answer Part 2: {func_to_run(2,day_data.input)}")

# Parsing helpers #
def aoc_lines(input) -> List[str]:
    return input.strip().splitlines()

def aoc_lists(input, type=int, delimiter=None) -> List[List[T]]:
    return [[type(item) for item in line.split(delimiter)] for line in aoc_lines(input)]

def aoc_grid(input, type=int) -> np.ndarray:
    return np.array([list(map(type, line)) for line in aoc_lines(input)], dtype=type)

def aoc_grid_inbounds(pos, grid) -> bool:
    return 0 <= pos[0] < grid.shape[0] and 0 <= pos[1] < grid.shape[1]

def aoc_keyvalue(input, delimiter=":", key_type=str, value_type=int) -> Dict[T,T]:
    return {
        key.strip(): value.strip() for key, value in (line.split(delimiter, 1) for line in aoc_lines(input))
        }


## Day 1

[Link to puzzle](https://adventofcode.com/2024/day/1)

In [74]:
# DAY 1 #
def run(part,i):
    left, right = zip(*(aoc_lists(i))) # Divide inputs into left and right columns

    def part_1():
        differences = [abs(l - r) for l, r in zip(sorted(left), sorted(right))]
        return sum(differences)

    def part_2():
        # Similarity - multiply each number in left list by the number of times it appears in the right list
        right_counts = Counter(right) # originally used right.count(), but this is significantly faster (13ms -> 1ms)
        return sum(value * right_counts[value] for value in left) 

    return part_1() if part == 1 else part_2()

day_data = init_day(day=1, test_solutions=(11, 31))
run_test(run, day_data)
run_real(run, day_data)

Test Part 1: Success!
Test Part 2: Success!
Answer Part 1: 2815556
CPU times: user 1.25 ms, sys: 256 µs, total: 1.5 ms
Wall time: 1.29 ms
Answer Part 2: 23927637
CPU times: user 1.04 ms, sys: 17 µs, total: 1.06 ms
Wall time: 1.06 ms


## Day 2

[Link to puzzle](https://adventofcode.com/2024/day/2)

In [75]:
# DAY 2 #
def run(part,i):
    def check(report):
        def is_safe(levels):
            return (
                # Safe if levels are asc/desc and no step is <1 or >3
                levels in (sorted(levels), sorted(levels, reverse=True))
                and all(1 <= abs(right - left) <= 3 for left, right in zip(levels, levels[1:])) # zip magic - this compares positions (0,1),(1,2),etc.
            )
        
        if is_safe(report): return True
        
        if part == 2: # Safe if any permutation with 1 step removed would be safe
            return any(is_safe(report[:pos] + report[pos + 1:]) for pos in range(len(report)))

        return False

    reports = aoc_lists(i)
    return sum(check(report) for report in reports)

day_data = init_day(day=2, test_solutions=(2, 4))
run_test(run, day_data)
run_real(run, day_data)

Test Part 1: Success!
Test Part 2: Success!
Answer Part 1: 585
CPU times: user 3.41 ms, sys: 164 µs, total: 3.58 ms
Wall time: 3.41 ms
Answer Part 2: 626
CPU times: user 7.6 ms, sys: 109 µs, total: 7.71 ms
Wall time: 7.72 ms


## Day 3

[Link to puzzle](https://adventofcode.com/2024/day/3)

In [108]:
# DAY 3 #
# and so the regex begins -_-
def run(part,i):
    def mul(input: str): # find all mul(), multiply, and then sum the results
        matches = re.findall(r"mul\((\d+),(\d+)\)", input) # regex for mul(digits,digits)
        return sum(int(a)*int(b) for a,b in matches)

    # in p2, anything between a don't() and the next do() is replaced with ""
    mulstring = i if part == 1 else re.sub(r"don't\(\).*?do\(\)", "", i, flags=re.DOTALL)
    return mul(mulstring)

day_data = init_day(day=3, test_solutions=(161, 48))
run_test(run, day_data)
run_real(run, day_data)

Test Part 1: Success!
Test Part 2: Success!
Answer Part 1: 170778545
CPU times: user 505 µs, sys: 0 ns, total: 505 µs
Wall time: 509 µs
Answer Part 2: 82868252
CPU times: user 366 µs, sys: 1 µs, total: 367 µs
Wall time: 369 µs


## Day 4

[Link to puzzle](https://adventofcode.com/2024/day/4)

In [193]:
# DAY 4 #
def run(part,i):
    grid = aoc_grid(i, type=str)
    cardinal_directions = [(0, 1),(0, -1),(1, 0),(-1, 0)]
    diagonal_directions = [(1, 1),(-1, -1),(1, -1),(-1, 1)]
    all_directions = cardinal_directions+diagonal_directions

    def find_matches(word, start, directions):
        total = 0
        for row, col in product(range(grid.shape[0]), range(grid.shape[1])):
            if grid[row, col] != start: continue

            # we are now at the start - check in all valid directions
            mas_count = 0
            for dx, dy in directions:

                # Part 1: Find # of 'word' in the grid
                if part == 1:
                    if all(
                        aoc_grid_inbounds(pos := (row+index*dx, col+index*dy), grid) and
                        grid[pos] == word[index]
                        for index in range(len(word))
                    ):
                        total += 1

                # Part 2: Find # of diagonally-intersecting 'MAS' in the grid
                elif part == 2:
                    if (
                        aoc_grid_inbounds(pos_m := (row + dx, col + dy), grid) and grid[pos_m] == 'M' and
                        aoc_grid_inbounds(pos_s := (row - dx, col - dy), grid) and grid[pos_s] == 'S' # MAS! (or SAM)
                    ):
                        mas_count += 1
                    if mas_count == 2:  # if we have 2 valid MAS, we have a cross (X-MAS)
                        total += 1; mas_count = 0
        return total

    word = 'XMAS' if part == 1 else ''
    start = word[0] if part == 1 else 'A'
    directions = all_directions if part == 1 else diagonal_directions
    return find_matches(word, start, directions)

day_data = init_day(day=4, test_solutions=(18, 9))
run_test(run, day_data)
run_real(run, day_data)

Test Part 1: Success!
Test Part 2: Success!
Answer Part 1: 2532
CPU times: user 107 ms, sys: 652 µs, total: 107 ms
Wall time: 108 ms
Answer Part 2: 1941
CPU times: user 43.4 ms, sys: 203 µs, total: 43.6 ms
Wall time: 43.7 ms


## Day 5

[Link to puzzle](https://adventofcode.com/2024/day/5)

In [233]:
# DAY 5 #
def run(part,i):
    rules, updates = i.split('\n\n')
    rules = aoc_lists(rules, int, "|")
    updates = aoc_lists(updates, int, ",")

    def add_middles(updates):
        return sum(update[len(update) // 2] for update in updates) # add up middle pages of each update in the list

    def is_valid_update(update):
        before = []; after = update[:]
        for page in update:
            for rule in rules:
                # check if any rules for this page are broken
                if (rule[0] == page and rule[1] in before) or (rule[1] == page and rule[0] in after):
                    return False
            before.append(after.pop(0))
        return True

    def fix_update(update):
        dependency_map = defaultdict(set) # will create empty set for a key if key doesn't exist
        for first_page, second_page in rules:
            if first_page in update and second_page in update:
                dependency_map[second_page].add(first_page) # add 1st page as a dependency (must come first) for 2nd page
        update.sort(key=lambda x: len(dependency_map[x])) # sort by # of dependencies
        return update
                
    correct = []
    for update in updates:
        if is_valid_update(update):
            if part == 1: # only look at correct updates
                correct.append(update)
        elif part == 2: # only look at incorrect updates (after fixing them)
            correct.append(fix_update(update))
        
    return add_middles(correct)

day_data = init_day(day=5, test_solutions=(143, 123))
run_test(run, day_data)
run_real(run, day_data)

Test Part 1: Success!
Test Part 2: Success!
Answer Part 1: 4924
CPU times: user 139 ms, sys: 1.91 ms, total: 141 ms
Wall time: 141 ms
Answer Part 2: 6085
CPU times: user 175 ms, sys: 1.53 ms, total: 177 ms
Wall time: 177 ms


## Day 6

[Link to puzzle](https://adventofcode.com/2024/day/6)

In [6]:
# DAY 6 #
def run(part,i):
    grid = aoc_grid(i, str)
    directions = [(0,-1),(-1,0),(0,1),(1,0)]

    def turn(dir):
        for ind,d in enumerate(directions):
            if d == dir: 
                return directions[ind+1 if ind < 3 else 0]

    def walk(pos, dir):
        visited = set()
        while aoc_grid_inbounds(pos, grid):
            if (pos,dir) in visited:
                return True # looping
            else: 
                visited.add((pos,dir))
            
            next_step = (pos[0]+dir[0], pos[1]+dir[1])
            if aoc_grid_inbounds(next_step, grid) and grid[next_step] == '#':
                dir = turn(dir)
            else:
                pos = next_step
            
        return {pos for pos, _ in visited} # no loop, return unique positions

    def find_loops(path):
        loops = 0
        for pos in path:
            if pos != guard_start:
                grid[pos] = '#'
                if walk(guard_start, dir) == True:
                    loops += 1
                grid[pos] = '.'
        return loops

    for row, grid_row in enumerate(grid):
        for col, char in enumerate(grid_row):
            if char == '^':
                guard_start = (row, col)
                dir = (-1,0) # in this problem, guard always starts facing up

    guard_path = walk(guard_start, dir)

    return len(guard_path) if part == 1 else find_loops(guard_path)

day_data = init_day(day=6, test_solutions=(41, 6))
run_test(run, day_data)
run_real(run, day_data)

Test Part 1: Success!
Test Part 2: Success!
Answer Part 1: 5564
CPU times: user 24.9 ms, sys: 474 µs, total: 25.4 ms
Wall time: 25.4 ms
Answer Part 2: 1976
CPU times: user 37.9 s, sys: 161 ms, total: 38.1 s
Wall time: 38.1 s


## Day 7

[Link to puzzle](https://adventofcode.com/2024/day/7)

In [ ]:
# DAY 7 #
def run(part,i):
    return 1

day_data = init_day(day=7, test_solutions=(0, 0))
run_test(run, day_data)
run_real(run, day_data)